In [1]:
from flask import Flask, render_template, request, session
from src.helper import download_embeddings
from langchain_pinecone import PineconeVectorStore
from langchain_groq import ChatGroq
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
from src.prompt import *
import os
import uuid

app = Flask(__name__)
app.secret_key = os.urandom(24).hex()  # Random secret key for sessions

load_dotenv()


c:\Users\Mungunsi\Desktop\Medical_chat_bot\Medical_chatbot_with_LLMs_LangChain_Pinecone_Flask_AWS\medibot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
# Initialize components
embeddings = download_embeddings()
index_name = "medicalchatbot"
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})
chat_model = ChatGroq(model="llama-3.3-70b-versatile")


C:\Users\Mungunsi\Desktop\Medical_chat_bot\Medical_chatbot_with_LLMs_LangChain_Pinecone_Flask_AWS\src\helper.py:50: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [3]:
# ============ MEMORY STORAGE ============
# Dictionary to store memory for each user session
session_memories = {}
# ========================================

In [4]:

def get_or_create_memory(session_id):
    """Get existing memory or create new one for session"""
    if session_id not in session_memories:
        session_memories[session_id] = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
    return session_memories[session_id]

In [5]:
@app.route('/')
def index():
    # Create unique session ID for each user
    if 'session_id' not in session:
        session['session_id'] = str(uuid.uuid4())
    
    # Initialize memory for this session
    get_or_create_memory(session['session_id'])
    
    return render_template('chat.html')

@app.route('/get', methods=["POST"])
def chat():
    try:
        # Get session ID
        session_id = session.get('session_id')
        if not session_id:
            return "Please refresh the page to start a new session."
        
        # Get user message
        msg = request.form['msg']
        print(f"\n📥 User ({session_id[:8]}): {msg}")
        
        # Get memory for this session
        memory = get_or_create_memory(session_id)
        
        # ============ MEMORY INTEGRATION ============
        # 1. Get conversation history from memory
        memory_vars = memory.load_memory_variables({})
        chat_history = memory_vars.get("chat_history", [])
        
        # 2. Create prompt WITH memory
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                MessagesPlaceholder(variable_name="chat_history"),  # Memory goes here
                ("human", "{input}"),
            ]
        )
        
        # 3. Create chains
        question_answering_chain = create_stuff_documents_chain(chat_model, prompt)
        rag_chain = create_retrieval_chain(retriever, question_answering_chain)
        
        # 4. Prepare input with memory
        chain_input = {
            "input": msg,
            "chat_history": chat_history  # Pass memory to chain
        }
        
        # 5. Get response
        response = rag_chain.invoke(chain_input)
        answer = response['answer']
        
        # 6. Save conversation to memory
        memory.save_context({"input": msg}, {"output": answer})
        # =============================================
        
        print(f"🤖 Bot: {answer[:100]}...")
        print(f"🧠 Memory now has {len(memory.chat_memory.messages)} messages")
        
        return answer
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return "I'm having trouble processing your request."

@app.route('/clear_memory', methods=['POST'])
def clear_memory():
    """Clear memory for current session"""
    session_id = session.get('session_id')
    if session_id and session_id in session_memories:
        session_memories[session_id].clear()
        return "Memory cleared!"
    return "No active session."

In [7]:

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.20.10.3:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

In [ ]:

chat(agent, "Nice! wish me luck!", thread_1)

"GOOD LUCK!\n\nMay your training sessions be productive, your endurance be strong, and your skills be sharp. May you stay focused, motivated, and injury-free.\n\nRemember to believe in yourself, trust your abilities, and have fun playing the game you love.\n\nYou've got this! Go out there and give it your all. I'll be rooting for you from afar.\n\nLet me know if you need any more advice or guidance. I'm here to help.\n\nGO CRUSH THAT GAME!"

In [ ]:
chat(agent, "My name is wilson i play as a center mid", thread_1)

"Nice to meet you, Wilson!\n\nAs a center midfielder, you're the heartbeat of the team, responsible for controlling the tempo of the game, distributing the ball, and creating scoring opportunities. It's a vital position that requires a great deal of energy, vision, and skill.\n\nI'm sure you'll be working hard to improve your passing range, your ability to read the game, and your endurance to cover the entire pitch. Remember to stay focused, communicate effectively with your teammates, and always be looking for ways to create space and exploit weaknesses in the opposition's defense.\n\nAs a center midfielder, you'll be involved in almost every play, so it's essential to stay hydrated, fueled, and mentally sharp throughout the game. Make sure to take care of yourself, both on and off the pitch, and you'll be ready to put in a top-notch performance.\n\nWhat's your favorite team or player, Wilson? Do you have any pre-game rituals or superstitions that help you get ready for a match?"

In [ ]:
chat(agent, "can you remind me of my name and position?", thread_1)

'Your name is **Wilson**, and you play as a **Center Midfielder**.'